<a href="https://colab.research.google.com/github/IlyasNasirov/notebook/blob/main/portfolio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy joylashgan qavat
- `max_levels` - ja'mi qavatlar soni
- `price` - uy narxi

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

In [2]:
df[df['size'].str.startswith('Площ')]

,location,district,rooms,size,level,max_levels,price
5347,"город Ташкент, Яшнободский район, Дархон",Яшнободский,4,Площадьземли:1сот,3,5,150000


In [3]:
df.loc[[5347]]

,location,district,rooms,size,level,max_levels,price
5347,"город Ташкент, Яшнободский район, Дархон",Яшнободский,4,Площадьземли:1сот,3,5,150000


In [4]:
df.loc[5347,'size']=10

In [5]:
df.size = df['size'].astype(dtype='float64')

In [6]:
import numpy as np
df.price = df.price.replace('Договорная',np.nan)

In [7]:
df['price'] = df['price'].astype(dtype='float64')

In [8]:
df['price'].mean()

np.float64(71334.21042057326)

In [9]:
df['price'] = df['price'].fillna(df.price.mean())

In [22]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [17]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
housing = train_set.drop('price',axis=1)
housing_labels = train_set['price'].copy()
housing_nums = housing.drop(['location','district'],axis=1)

In [23]:
num_attribs = list(housing_nums)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attribs),
    ('cat', OrdinalEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

In [25]:
LR_model = LinearRegression()
LR_model.fit(housing_prepared,housing_labels)

LinearRegression()

In [26]:
test_data = housing.sample(7)
test_data.drop('location',axis=1,inplace=True)
test_labels = housing_labels.loc[test_data.index]
test_labels

,price
3847,37000.0
3242,29500.0
2535,31000.0
1776,280000.0
2231,37500.0
3827,55000.0
6313,57200.0


In [27]:
test_data_prepared = full_pipeline.transform(test_data)

In [28]:
predicted_data = LR_model.predict(test_data_prepared)

In [29]:
pd.DataFrame({'Predicted':predicted_data,'Real':test_labels})

,Predicted,Real
3847,58785.544318,37000.0
3242,42385.671938,29500.0
2535,46131.209070,31000.0
1776,104642.846956,280000.0
2231,57062.231790,37500.0
3827,92138.447657,55000.0
6313,58565.981532,57200.0
